<a href="https://colab.research.google.com/github/shalomorlando/machine_learning_tasks/blob/main/ML_Group_Final_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   Shalom Otieno 111531
2.   Mutabari Fridah 111033
3.   Joshua Mochama 106272
4.   Catherine Muigai 113231



#Import necessary modules

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler , PowerTransformer
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV
import encodings

# Importing the data file and preparing it

In [5]:
data=pd.read_csv('/content/Housing Pricing.csv')

In [6]:
data.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500


In [7]:
data.shape

(1460, 77)

In [8]:
#statistic description of the dataset columns
data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [9]:
#since id does not provide information relevant to our problem, we drop it
data.drop('Id',inplace=True, axis=1)

In [10]:
#obtain a list of original features for tracking purposes
original_features=list(data.columns)

In [11]:
'''
  description from pandas docs

  pandas.dataframe.isna() - Detects missing values. Return a boolean same-sized 
  object indicating if the values are NA. Returns a DataFrame. Mask of bool 
  values for each element in DataFrame that indicates whether an element is an 
  NA value.

  pandas.dataframe.any()- Returns whether any element is True, potentially over 
  an axis. Returns False unless there is at least one element within a series or
  along a Dataframe axis that is True or equivalent(e.g. non-zero or non-empty).
  Returns series or dataframe.

'''
#data.isna()
data.isna().any()

MSSubClass       False
MSZoning         False
LotFrontage       True
LotArea          False
Street           False
                 ...  
MoSold           False
YrSold           False
SaleType         False
SaleCondition    False
SalePrice        False
Length: 76, dtype: bool

# Dealing with missing values

In [12]:
#obtain columns with missing values
features_missing_values=list(data.columns[data.isna().any()])

In [13]:
bfill, mean_fill=[],[]

In [14]:
'''
  separate the features with missing values according to the method to be used in
  filling them.
'''
for feature in features_missing_values:
  print(feature, '', data[feature].isna().sum()/len(data)*100, '',data[feature].dtype)
  if data[feature].dtype== 'float64':
    mean_fill.append(feature)
  else:
    bfill.append(feature)

LotFrontage  17.73972602739726  float64
MasVnrType  0.547945205479452  object
MasVnrArea  0.547945205479452  float64
BsmtQual  2.5342465753424657  object
BsmtCond  2.5342465753424657  object
BsmtExposure  2.6027397260273974  object
BsmtFinType1  2.5342465753424657  object
BsmtFinType2  2.6027397260273974  object
Electrical  0.0684931506849315  object
FireplaceQu  47.26027397260274  object
GarageType  5.5479452054794525  object
GarageYrBlt  5.5479452054794525  float64
GarageFinish  5.5479452054794525  object
GarageQual  5.5479452054794525  object
GarageCond  5.5479452054794525  object


In [15]:
mean_fill

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [16]:
bfill

['MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [17]:
'''
  use the mean for each column to fill the missing values
  method used -> dataframe.fillna(dataframe[feature].mean())
'''
for feature in mean_fill:
  data[feature].fillna(data[feature].mean(),inplace=True)

In [18]:
'''
  Fill missing values using backward filling. i.e use value of previous row to 
  fill current.
  method: dataframe[feature].fillna(method='bfill') 
'''
for feature in bfill:
  data[feature].fillna(method='bfill', inplace=True)

In [19]:
#check for any other missing values in the dataframe
data.columns[data.isna().any()]

Index(['FireplaceQu'], dtype='object')

In [20]:
data ['FireplaceQu']. isna().sum()

2

In [21]:
data['FireplaceQu'].fillna(method='ffill', inplace=True)

# Data encoding



In [22]:
'''
  determine the types of non-numerical features 
'''
categorical_features=list(data.select_dtypes(include=['object']).columns)
categorical_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [23]:
'''
  Obtain the numerical features using set subtraction
'''
numerical=list(set(original_features)-set(categorical_features))
numerical

['PoolArea',
 'BedroomAbvGr',
 'BsmtFinSF2',
 'YearBuilt',
 'ScreenPorch',
 'HalfBath',
 'LotFrontage',
 'TotalBsmtSF',
 'BsmtUnfSF',
 'TotRmsAbvGrd',
 'YrSold',
 'LotArea',
 'GarageArea',
 'MSSubClass',
 'BsmtFullBath',
 'MoSold',
 '1stFlrSF',
 'MiscVal',
 'MasVnrArea',
 'Fireplaces',
 '2ndFlrSF',
 'LowQualFinSF',
 'GarageCars',
 'GrLivArea',
 'YearRemodAdd',
 'GarageYrBlt',
 'WoodDeckSF',
 'EnclosedPorch',
 'OverallCond',
 'OverallQual',
 'OpenPorchSF',
 'KitchenAbvGr',
 'BsmtHalfBath',
 'FullBath',
 'SalePrice',
 'BsmtFinSF1',
 '3SsnPorch']

In [24]:
'''
  determine the data scales
'''
nominal=['MSZoning','LandContour','Neighborhood']
ordinal=list(set(categorical_features)-set(nominal))

In [25]:
target=['SalePrice']

In [26]:
#Convert categorical variables into dummy/indicator variables.
df_nominal=pd.get_dummies(data[nominal])

In [27]:
for feature in ordinal:
  #Return Series of codes as well as the index.
  data[feature]=data[feature].astype('category').cat.codes

df_ordinal=data[ordinal]

In [28]:
data[numerical]

,PoolArea,BedroomAbvGr,BsmtFinSF2,YearBuilt,ScreenPorch,HalfBath,LotFrontage,TotalBsmtSF,BsmtUnfSF,TotRmsAbvGrd,YrSold,LotArea,GarageArea,MSSubClass,BsmtFullBath,MoSold,1stFlrSF,MiscVal,MasVnrArea,Fireplaces,2ndFlrSF,LowQualFinSF,GarageCars,GrLivArea,YearRemodAdd,GarageYrBlt,WoodDeckSF,EnclosedPorch,OverallCond,OverallQual,OpenPorchSF,KitchenAbvGr,BsmtHalfBath,FullBath,SalePrice,BsmtFinSF1,3SsnPorch
0,0,3,0,2003,0,1,65.0,856,150,8,2008,8450,548,60,1,2,856,0,196.0,0,854,0,2,1710,2003,2003.0,0,0,5,7,61,1,0,2,208500,706,0
1,0,3,0,1976,0,0,80.0,1262,284,6,2007,9600,460,20,0,5,1262,0,0.0,1,0,0,2,1262,1976,1976.0,298,0,8,6,0,1,1,2,181500,978,0
2,0,3,0,2001,0,1,68.0,920,434,6,2008,11250,608,60,1,9,920,0,162.0,1,866,0,2,1786,2002,2001.0,0,0,5,7,42,1,0,2,223500,486,0
3,0,3,0,1915,0,0,60.0,756,540,7,2006,9550,642,70,1,2,961,0,0.0,1,756,0,3,1717,1970,1998.0,0,272,5,7,35,1,0,1,140000,216,0
4,0,4,0,2000,0,1,84.0,1145,490,9,2008,14260,836,60,1,12,1145,0,350.0,1,1053,0,3,2198,2000,2000.0,192,0,5,8,84,1,0,2,250000,655,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,3,0,1999,0,1,62.0,953,953,7,2007,7917,460,60,0,8,953,0,0.0,1,694,0,2,1647,2000,1999.0,0,0,5,6,40,1,0,2,175000,0,0
1456,0,3,163,1978,0,0,85.0,1542,589,7,2010,13175,500,20,1,2,2073,0,119.0,2,0,0,2,2073,1988,1978.0,349,0,6,6,0,1,0,2,210000,790,0
1457,0,4,0,1941,0,0,66.0,1152,877,9,2010,9042,252,70,0,5,1188,2500,0.0,2,1152,0,1,2340,2006,1941.0,0,0,9,7,60,1,0,2,266500,275,0
1458,0,2,1029,1950,0,0,68.0,1078,0,5,2010,9717,240,20,1,4,1078,0,0.0,0,0,0,1,1078,1996,1950.0,366,112,6,5,0,1,0,1,142125,49,0


In [29]:
#create new dataframe by concatinating all the modified sub-sections
new_data=pd.concat([df_nominal,df_ordinal,data[numerical]],axis=1)
new_data

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BsmtExposure,HouseStyle,Condition1,HeatingQC,RoofStyle,GarageCond,...,ExterQual,BsmtFinType1,CentralAir,PoolArea,BedroomAbvGr,BsmtFinSF2,YearBuilt,ScreenPorch,HalfBath,LotFrontage,TotalBsmtSF,BsmtUnfSF,TotRmsAbvGrd,YrSold,LotArea,GarageArea,MSSubClass,BsmtFullBath,MoSold,1stFlrSF,MiscVal,MasVnrArea,Fireplaces,2ndFlrSF,LowQualFinSF,GarageCars,GrLivArea,YearRemodAdd,GarageYrBlt,WoodDeckSF,EnclosedPorch,OverallCond,OverallQual,OpenPorchSF,KitchenAbvGr,BsmtHalfBath,FullBath,SalePrice,BsmtFinSF1,3SsnPorch
0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5,2,0,1,4,...,2,2,1,0,3,0,2003,0,1,65.0,856,150,8,2008,8450,548,60,1,2,856,0,196.0,0,854,0,2,1710,2003,2003.0,0,0,5,7,61,1,0,2,208500,706,0
1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,1,0,1,4,...,3,0,1,0,3,0,1976,0,0,80.0,1262,284,6,2007,9600,460,20,0,5,1262,0,0.0,1,0,0,2,1262,1976,1976.0,298,0,8,6,0,1,1,2,181500,978,0
2,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,5,2,0,1,4,...,2,2,1,0,3,0,2001,0,1,68.0,920,434,6,2008,11250,608,60,1,9,920,0,162.0,1,866,0,2,1786,2002,2001.0,0,0,5,7,42,1,0,2,223500,486,0
3,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5,2,2,1,4,...,3,0,1,0,3,0,1915,0,0,60.0,756,540,7,2006,9550,642,70,1,2,961,0,0.0,1,756,0,3,1717,1970,1998.0,0,272,5,7,35,1,0,1,140000,216,0
4,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,2,0,1,4,...,2,2,1,0,4,0,2000,0,1,84.0,1145,490,9,2008,14260,836,60,1,12,1145,0,350.0,1,1053,0,3,2198,2000,2000.0,192,0,5,8,84,1,0,2,250000,655,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5,2,0,1,4,...,3,5,1,0,3,0,1999,0,1,62.0,953,953,7,2007,7917,460,60,0,8,953,0,0.0,1,694,0,2,1647,2000,1999.0,0,0,5,6,40,1,0,2,175000,0,0
1456,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,2,2,4,1,4,...,3,0,1,0,3,163,1978,0,0,85.0,1542,589,7,2010,13175,500,20,1,2,2073,0,119.0,2,0,0,2,2073,1988,1978.0,349,0,6,6,0,1,0,2,210000,790,0
1457,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5,2,0,1,4,...,0,2,1,0,4,0,1941,0,0,66.0,1152,877,9,2010,9042,252,70,0,5,1188,2500,0.0,2,1152,0,1,2340,2006,1941.0,0,0,9,7,60,1,0,2,266500,275,0
1458,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,3,4,...,3,2,1,0,2,1029,1950,0,0,68.0,1078,0,5,2010,9717,240,20,1,4,1078,0,0.0,0,0,0,1,1078,1996,1950.0,366,112,6,5,0,1,0,1,142125,49,0


In [30]:
new_data.shape

(1460, 107)

# Standardization

In [31]:
#Drop the  target variable before creating the feature matrix.
new_data.drop('SalePrice',inplace=True,axis=1)

In [32]:
#converting the pandas dataframe into a numpy array (feature matrix ).
X=new_data.to_numpy()

In [33]:
#Checking what is contained in x
X

array([[  0.,   0.,   0., ...,   2., 706.,   0.],
       [  0.,   0.,   0., ...,   2., 978.,   0.],
       [  0.,   0.,   0., ...,   2., 486.,   0.],
       ...,
       [  0.,   0.,   0., ...,   2., 275.,   0.],
       [  0.,   0.,   0., ...,   1.,  49.,   0.],
       [  0.,   0.,   0., ...,   1., 830.,   0.]])

In [34]:
#standardizing the features by the mean and scalling to unit variance.
X=StandardScaler().fit_transform(X)

In [35]:
#Checking the values in X after standardization.
X[0]

array([-0.08304548, -0.21585871, -0.10526316,  0.51813339, -0.41895507,
       -0.21235968, -0.18831089, -0.15899968,  0.33712564, -0.10854037,
       -0.03703704, -0.10526316, -0.20339487, -0.1398323 ,  2.95522137,
       -0.19025216, -0.27116307, -0.23917551, -0.16124951, -0.10854037,
       -0.1863522 , -0.42683279, -0.07875671, -0.22941573, -0.16998114,
       -0.23595776, -0.28963792, -0.13199092, -0.23106504, -0.20521398,
       -0.25018188, -0.13199092, -0.16347148, -0.08712888,  0.64046232,
        1.02668924, -0.03628912, -0.88465767, -0.49151573,  0.18797343,
       -1.23804229, -0.12579688, -0.12304604, -0.71775144, -0.40936914,
       -0.29055229, -0.23012211,  0.60466978, -0.22571613,  0.36420746,
       -0.03174026,  0.06423821,  0.303692  ,  0.31386709,  0.74309239,
        0.28974476,  0.75073056, -0.02618016,  0.25597761,  0.2085023 ,
        0.83559903,  1.11993319,  0.22403433,  0.75162623,  0.30941909,
        0.28780036, -0.41169079, -0.77797579, -0.40779461,  0.26

In [36]:
#Refreshing the target variable y.
y=data[target]
y

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000
...,...
1455,175000
1456,210000
1457,266500
1458,142125


In [37]:
#Changing the target matrix to a numpy array
y=y.to_numpy()

In [38]:

y.shape

(1460, 1)

In [39]:
y

array([[208500],
       [181500],
       [223500],
       ...,
       [266500],
       [142125],
       [147500]])

In [40]:
#standardizing the features by the mean and scalling to unit variants.
y=StandardScaler().fit_transform(y)

In [41]:
y

array([[ 0.34727322],
       [ 0.00728832],
       [ 0.53615372],
       ...,
       [ 1.07761115],
       [-0.48852299],
       [-0.42084081]])

In [42]:
X.shape

(1460, 106)

# Train a Dense Feed forward neural network
### We first split the dataset into train / test splits
### Then we use Tensorflow.keras to build sequential model that uses keras.regularizers.l1


### Alternate method provided at the towards the end

In [43]:
from sklearn.model_selection import train_test_split
#spliting the dataset
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [44]:
X_train.shape

(1022, 106)

In [45]:
Y_train.shape

(1022, 1)

## Model + regularization

In [46]:
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
from keras.optimizers import Adam

In [47]:
model = Sequential()
model.add(Dense(106, kernel_regularizer=regularizers.l1(0.01), activation="relu", input_shape=(106,)))
model.add(Dense(32, kernel_regularizer=regularizers.l1(0.01), activation="relu"))
model.add(Dense(8, kernel_regularizer=regularizers.l1(0.01), activation="relu"))
model.add(Dense(1)) # final layer has no activation because this is a regression problem

In [48]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['accuracy'])

In [49]:
history = model.fit(X_train, Y_train,
          batch_size=100, epochs=400
          )

Epoch 1/400
11/11 [==============================] - 14s 3ms/step - loss: 14.0332 - accuracy: 0.0000e+00
Epoch 2/400
11/11 [==============================] - 0s 2ms/step - loss: 12.5956 - accuracy: 0.0000e+00
Epoch 3/400
11/11 [==============================] - 0s 2ms/step - loss: 11.2915 - accuracy: 0.0000e+00
Epoch 4/400
11/11 [==============================] - 0s 2ms/step - loss: 10.1982 - accuracy: 0.0000e+00
Epoch 5/400
11/11 [==============================] - 0s 2ms/step - loss: 9.0753 - accuracy: 0.0000e+00
Epoch 6/400
11/11 [==============================] - 0s 2ms/step - loss: 8.0342 - accuracy: 0.0000e+00
Epoch 7/400
11/11 [==============================] - 0s 2ms/step - loss: 7.1169 - accuracy: 0.0000e+00
Epoch 8/400
11/11 [==============================] - 0s 2ms/step - loss: 6.2051 - accuracy: 0.0000e+00
Epoch 9/400
11/11 [==============================] - 0s 2ms/step - loss: 5.3869 - accuracy: 0.0000e+00
Epoch 10/400
11/11 [==============================] - 0s 2ms/step - 

In [50]:
model.evaluate(X_test,Y_test)

14/14 [==============================] - 0s 1ms/step - loss: 0.2007 - accuracy: 0.0000e+00


[0.20072533190250397, 0.0]

In [51]:
model.evaluate(X_test, Y_test)[0]

14/14 [==============================] - 0s 1ms/step - loss: 0.2007 - accuracy: 0.0000e+00


0.20072533190250397

#Alternative L1 Based Feature Selection using LassoCV

In [52]:
#will be used to select feature
from sklearn.feature_selection import SelectFromModel

In [53]:
regressor = LassoCV()
regressor.fit(X,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [54]:
selector = SelectFromModel(regressor, prefit=True)

In [56]:
X_new = selector.transform(X)
X_new.shape

(1460, 72)

In [57]:
X_train2,X_test2,Y_train2,Y_test2=train_test_split(X_new,y,test_size=0.3)

In [58]:
X_train2.shape

(1022, 72)

In [59]:
Y_train2.shape

(1022, 1)

In [60]:
model_2 = Sequential([
    Dense(72, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1),
])

In [61]:
model_2.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])

In [62]:
history=model_2.fit(X_train2, Y_train2,epochs=100)

Epoch 1/100
32/32 [==============================] - 8s 2ms/step - loss: 0.6953 - accuracy: 0.0000e+00
Epoch 2/100
32/32 [==============================] - 0s 1ms/step - loss: 0.3193 - accuracy: 0.0000e+00
Epoch 3/100
32/32 [==============================] - 0s 1ms/step - loss: 0.2322 - accuracy: 0.0000e+00
Epoch 4/100
32/32 [==============================] - 0s 1ms/step - loss: 0.2068 - accuracy: 0.0000e+00
Epoch 5/100
32/32 [==============================] - 0s 1ms/step - loss: 0.1782 - accuracy: 0.0000e+00
Epoch 6/100
32/32 [==============================] - 0s 1ms/step - loss: 0.1683 - accuracy: 0.0000e+00
Epoch 7/100
32/32 [==============================] - 0s 1ms/step - loss: 0.1504 - accuracy: 0.0000e+00
Epoch 8/100
32/32 [==============================] - 0s 1ms/step - loss: 0.1456 - accuracy: 0.0000e+00
Epoch 9/100
32/32 [==============================] - 0s 1ms/step - loss: 0.1386 - accuracy: 0.0000e+00
Epoch 10/100
32/32 [==============================] - 0s 1ms/step - loss:

In [63]:
model_2.evaluate(X_test2,Y_test2)

14/14 [==============================] - 0s 2ms/step - loss: 0.2652 - accuracy: 0.0000e+00


[0.26522013545036316, 0.0]